In [40]:
import warnings
import pickle
import copy
import torch
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from model import BertClassifier
from console1 import generate_partition
from console1 import Hypercube_wpart
from console1 import make_cube_list
from console1 import single_subset_norm
from console1 import two_subset_norm


F_P = {
    0: [0,1],
    1: [2,3],
    2: [4,5,6,7,8,9,10,11],
    3: [12,13],
}


In [4]:
df_10 = pd.read_csv('10instance.csv',index_col='index')
df_10 = df_10.drop(columns=['Output','DFT'])
background = pd.read_csv('background_dataset_2.csv',index_col = 'Unnamed: 0')
background = background.drop(columns=['Output'])
df = pd.read_csv('data_with_output.csv')
df = df.drop(columns=['Output'])
print(df_10.head(3))
print(background.head(3))
print(df.head(3))

       X_-TACT_TIME_mean  X_-CONVEYOR_SPEED_mean  PUMP_high  PUMP_low  \
index                                                                   
1                     90                    4200   37183.04  12891.48   
2                     90                    4200   40499.96  12198.94   
3                     90                    4200   35524.30  12731.66   

       CLN1_over-etching-ratio  CLN1_EPT_time  clean_count  \
index                                                        
1                     0.003292          10023            2   
2                     0.005368          10059            3   
3                     0.008401           9999            3   

       EPT_clean_count_ratio  NH3_TREAT_-RF_FREQ-max  \
index                                                  
1                     5011.5                   13963   
2                     3353.0                   13948   
3                     3333.0                   13979   

       NH3_TREAT_-RF_FREQ-range  NH3_TREAT

In [5]:
mean1 = df.mean()
std1 = df.std()
print(mean1)
print(std1)

X_-TACT_TIME_mean             154.203192
X_-CONVEYOR_SPEED_mean       3132.278651
PUMP_high                   42443.758562
PUMP_low                    11360.190378
CLN1_over-etching-ratio         0.004456
CLN1_EPT_time               11845.178373
clean_count                     3.684303
EPT_clean_count_ratio        4821.724736
NH3_TREAT_-RF_FREQ-max      13901.145963
NH3_TREAT_-RF_FREQ-range      383.961172
NH3_TREAT_-RF_FREQ-mean     13623.735749
NP_3_-MFC_VOL_SIH4-range        1.005759
VENT_high                   16660.791956
VENT_low                     7010.155928
dtype: float64
X_-TACT_TIME_mean              14.183177
X_-CONVEYOR_SPEED_mean        352.664542
PUMP_high                   12038.304024
PUMP_low                     1858.782605
CLN1_over-etching-ratio         0.004441
CLN1_EPT_time                1218.368666
clean_count                     2.017230
EPT_clean_count_ratio        3528.616445
NH3_TREAT_-RF_FREQ-max        146.644850
NH3_TREAT_-RF_FREQ-range      138.132554
N

In [6]:
c,_,_ = generate_partition(F_P)
collist = [i.tolist() for i in c]
colname = [tuple(i) for i in collist]
vertex_df = pd.DataFrame(columns=colname)
print(vertex_df.shape,vertex_df)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
s1_model_path = 'stage_1_checkpoint.pth'
s1_model =  torch.load(s1_model_path, map_location=device).to(device)
s1_model.eval()
json_file_path = 'controllable_para_v014_14.json'
tool_name = 'ASCVD'
with open(json_file_path, 'r') as f:
    params = json.load(f)[tool_name]
    print(params)
    f.close()
target_features = []

''' 
    The feature_translation is a list of tuples, each tuple contains two integers.
    The tuples record the correponding position of the ith feature in the 4*freature matrix
    as the input the prediction model.
    Ex.
    [(0, 0), (0, 1), 
     (1, 2), (1, 3), 
     (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), 
     (3, 12),(3, 13)]
'''
# Flat the feature list and construct teh freature translation list to map feature into the input of the model
feature_translation = []
sub_op_num = 0
for entry in [sub_op for _, sub_op in params.items()]:
    if isinstance(entry, str):
        target_features.append(entry)
    else:
        target_features.extend(entry)
        feature_translation.extend([(sub_op_num,len(feature_translation)+j) for j in range(len(entry))])
        sub_op_num = sub_op_num+1 

print(target_features)
print(f"There are {sub_op_num} sub processes. So the feature translation is {feature_translation}")

def padding_zero(df, flag): 
    # 將一維參數matrix擴展為4維
    data_arr = df.to_numpy()
    result = []
    for data in data_arr:
        empty_arr = np.zeros((sub_op_num, len(feature_translation))) # chamber數 * 總參數數量
        for i, pos in enumerate(feature_translation):
            empty_arr[pos[0]][pos[1]] = data[i]
        if(flag == 1): # bert.py使用
            result.append(empty_arr)
        if(flag == 2): # bert_du.py使用
            result.append(empty_arr.tolist())
    
    if(flag == 1): # bert.py使用
        result = pd.DataFrame({'X': [result[i] for i in range(len(result))]})
    return result

print(device)

(0, 16) Empty DataFrame
Columns: [(), (0, 1), (2, 3), (4, 5, 6, 7, 8, 9, 10, 11), (12, 13), (0, 1, 2, 3), (0, 1, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 12, 13), (2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (2, 3, 12, 13), (4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 2, 3, 12, 13), (0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)]
Index: []
{'EQ': ['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean'], 'PUMP': ['PUMP_high', 'PUMP_low'], 'CH': ['CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', 'NH3_TREAT_-RF_FREQ-mean', 'NP_3_-MFC_VOL_SIH4-range'], 'VENT': ['VENT_high', 'VENT_low'], 'y': 'Output'}
['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean', 'PUMP_high', 'PUMP_low', 'CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', '

In [7]:
def std_model_inference(data):  #standardize dataframe
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    s1_model.eval()
    print(data.head(3))   
    
    data_4d = padding_zero(data,flag=1)
    
    #print(f'4D data:\n{data_4d.head()}')  # Debug
    
    data_4d_array = np.array([e for entry in data_4d.values for e in entry])
    data_4d_tensor = torch.tensor(data_4d_array,dtype=torch.float)
    
    #print(f'Tensor data shape: {data_4d_tensor.shape}')  # Debug 
    
    my_dataset = TensorDataset(data_4d_tensor)
    batch_size = min(256, int(data_4d_tensor.size()[0]))
    my_loader = DataLoader(my_dataset, batch_size=batch_size,num_workers=4)
    data_output = []
    with torch.no_grad():
        for batch_data in my_loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
            batch_data = batch_data[0].to(device)
        # 将数据传递给模型进行推理
            batch_output = s1_model(batch_data)
            probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
            data_output += probs
    data_output_array = np.array([output.cpu().numpy()[0] for output in data_output])
    data_expectation_value = data_output_array.mean()
    return data_expectation_value, data_output_array


def model_inference(data):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    scaler = StandardScaler()
    if 'Output' in data.columns:
        data = data.drop(columns=['Output'])
    data_standardized_df = pd.DataFrame(scaler.fit_transform(data))
    data_4d = padding_zero(data_standardized_df,flag=1)
    data_4d_array = np.array([e for entry in data_4d.values for e in entry])
    data_4d_tensor = torch.tensor(data_4d_array,dtype=torch.float)
    my_dataset = TensorDataset(data_4d_tensor)
    batch_size = min(256, int(data_4d_tensor.size()[0]))
    my_loader = DataLoader(my_dataset, batch_size=batch_size,num_workers=4)
    data_output = []
    with torch.no_grad():
        for batch_data in my_loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
            batch_data = batch_data[0].to(device)
        # 将数据传递给模型进行推理
            batch_output = s1_model(batch_data)
            probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
            data_output += probs
    data_output_array = np.array([output.cpu().numpy()[0] for output in data_output])
    data_expectation_value = data_output_array.mean()
    return data_expectation_value, data_output_array

In [8]:
warnings.filterwarnings("ignore", category=FutureWarning)
vertices_fl = 'E_vi_20240610.pkl'
AUO_coalitions, _, _ = generate_partition(F_P) 
count_n = 0

In [11]:
for i_index in range(10):
    print(f'index = {i_index}')
    instance = df_10.iloc[i_index]
    std_instance = (instance - mean1) / std1
    coalition_estimated_values = {}
    #print(f' std_instance = {std_instance}')
    eval_df = background
    std_eval_df = (background - mean1) / std1
    base_mean, details = std_model_inference(std_eval_df)
    #print(f'Base mean: {base_mean}')
    for coalition in AUO_coalitions: 
        vi_df = std_eval_df.copy()  #用copy()才不會去更改到原始的dataframe
        if len(coalition)!=0:
            vi_df.iloc[:,coalition] = std_instance.iloc[coalition]          
        print(f'coalition = {coalition}')
        #print(vi_df.head(3))
        exp, details = std_model_inference(vi_df)
        #print(f'exp = {exp}')
        coalition_estimated_values[tuple(coalition)] =  exp - base_mean
        #print(f'coalition_estimated_values[tuple(coalition)] = {coalition_estimated_values[tuple(coalition)]}')
        count_n += 1  
        if count_n % 10 == 0:
            with open(vertices_fl, 'wb') as f:
                pickle.dump(vertex_df, f)
            print(f"Saved {count_n} vertex")
    #print(f'coalition_estimated_values = {coalition_estimated_values}')
    instance_df = pd.DataFrame([coalition_estimated_values])
    vertex_df = pd.concat([vertex_df, instance_df], ignore_index=True)            
   

with open(vertices_fl, 'wb') as f:
    pickle.dump(vertex_df, f)

index = 0
       X_-TACT_TIME_mean  X_-CONVEYOR_SPEED_mean  PUMP_high  PUMP_low  \
33414          -0.296351                0.050250  -1.253646 -0.482215   
6315            0.408710               -1.367528   0.964054 -0.293886   
1647           -0.296351                0.050250  -0.089581 -0.461749   

       CLN1_over-etching-ratio  CLN1_EPT_time  clean_count  \
33414                -0.721876       0.131998     0.652230   
6315                  2.970472      -1.490664     0.652230   
1647                 -0.733209      -1.507900    -0.834958   

       EPT_clean_count_ratio  NH3_TREAT_-RF_FREQ-max  \
33414              -0.685970                0.517264   
6315               -0.798025                0.646828   
1647                0.051656                0.537721   

       NH3_TREAT_-RF_FREQ-range  NH3_TREAT_-RF_FREQ-mean  \
33414                  0.499801                 0.302462   
6315                   0.463604                 0.737867   
1647                   0.420168            

In [49]:
vertex_df.index = pd.Index(range(10))

In [50]:
datalist = make_cube_list(vertex_df, F_P)
pgd_df, r_df, scn_df = single_subset_norm(datalist, F_P)

0    0.268390
1    0.525856
2    0.394537
3    0.610870
dtype: float64


In [51]:
print("partial gradient:")
print(pgd_df)
print("residual:")
print(r_df)
print("scaled norm:")
print(scn_df)

partial gradient:
          0         1         2         3
0  0.500838  0.115460  0.333158  0.137761
1  0.303936  0.147803  0.366412  0.073140
2  0.308178  0.059482  0.105104  0.063555
3  0.671913  0.066881  0.209693  0.079837
4  0.459564  0.071586  0.129845  0.030042
5  0.294359  0.041097  1.142473  0.020852
6  0.496064  0.111332  0.152196  0.046776
7  0.451388  0.093220  0.102837  0.078490
8  0.282263  0.048968  1.128993  0.029767
9  0.401971  0.119164  0.122864  0.112004
residual:
          0         1         2         3
0  0.086977  0.057886  0.067200  0.088059
1  0.148439  0.084971  0.138213  0.053123
2  0.087073  0.041365  0.074096  0.038975
3  0.117159  0.049635  0.111819  0.052923
4  0.056022  0.039665  0.040808  0.020485
5  0.136879  0.012665  0.136931  0.012971
6  0.066615  0.040167  0.066935  0.034230
7  0.082399  0.064308  0.059597  0.029816
8  0.137426  0.014110  0.137154  0.016764
9  0.070074  0.064792  0.067979  0.054687
scaled norm:
          0         1         2    

In [55]:
vertex_df.to_csv('10instance_raw.csv', index=False)

In [54]:
vertex_df

,(),"(0, 1)","(2, 3)","(4, 5, 6, 7, 8, 9, 10, 11)","(12, 13)","(0, 1, 2, 3)","(0, 1, 4, 5, 6, 7, 8, 9, 10, 11)","(0, 1, 12, 13)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","(2, 3, 12, 13)","(4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","(0, 1, 2, 3, 12, 13)","(0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)"
0,0.0,0.155535,-0.010374,-0.085595,-0.024421,0.135363,0.056374,0.161163,-0.097013,-0.041486,-0.163453,0.011295,0.149928,0.092972,-0.194775,-0.003234
1,0.0,0.155535,-0.009950,-0.054616,0.020290,0.141272,0.015947,0.155482,-0.063358,0.009653,-0.026907,-0.049967,0.128872,0.027994,-0.031373,-0.099604
2,0.0,0.155535,-0.000844,0.011546,0.000894,0.133285,0.119925,0.139673,0.035556,-0.000988,0.019490,0.106507,0.108694,0.086382,0.047812,0.063568
3,0.0,0.155535,0.003487,-0.110151,-0.049413,0.159779,0.140833,0.145138,-0.138458,-0.053916,-0.109430,0.160772,0.161854,0.138468,-0.139686,0.183511
4,0.0,0.155535,-0.029717,-0.023518,-0.018372,0.155713,0.085068,0.158429,-0.063635,-0.044645,-0.028191,0.095306,0.150578,0.098137,-0.069919,0.103981
5,0.0,0.155535,-0.014853,-0.329010,0.014576,0.136593,-0.311704,0.156713,-0.335085,-0.001809,-0.327938,-0.327367,0.135295,-0.315931,-0.329839,-0.325756
6,0.0,0.155535,-0.020856,-0.038206,0.017256,0.141932,0.150388,0.148470,-0.107440,-0.001680,-0.043927,0.125667,0.126153,0.145188,-0.096951,0.093205
7,0.0,0.155535,0.026192,-0.057218,0.023124,0.137265,0.171963,0.164149,-0.022826,0.056437,-0.033531,0.126318,0.155541,0.171570,0.023015,0.166818
8,0.0,0.155535,-0.006117,-0.310767,0.019851,0.136254,-0.304989,0.161524,-0.331208,0.013685,-0.305932,-0.321080,0.136146,-0.306904,-0.325371,-0.322777
9,0.0,0.155535,0.018429,-0.054660,-0.006771,0.135212,0.066106,0.183676,0.032272,0.017634,-0.021572,0.104844,0.181464,0.141849,0.038138,0.156815


In [42]:
import pickle

# Define the path to your .pkl file
file_path = 'E_vi_20240610.pkl'

# Read the .pkl file
with open(file_path, 'rb') as file:  # 'rb' means read binary mode
    data10 = pickle.load(file)

data10 = data10.iloc[1:]

In [44]:
type(vertex_df)

pandas.core.frame.DataFrame

In [45]:
type(data10)

pandas.core.frame.DataFrame

In [46]:
vertex_df = data10

In [47]:
datalist = make_cube_list(vertex_df, F_P)
pgd_df, r_df, scn_df = single_subset_norm(vertex_df, F_P)

KeyError: 0